In [56]:
import pandas as pd
import numpy as np

In [66]:
pd.set_option('display.max_row', 50)
pd.set_option('display.max_column', 120)

# Load & merge

### 1. Prepping

In [57]:
df_bigtable = pd.read_csv('data/ailes_avion.csv', usecols=['Name', 'Family', 'Thickness(%)', 'x-Location_of_Max_Thk(%)', 'Camber(%)'])
df_index = pd.read_csv('data/index_matchmaking.csv', usecols=['Name_big', 'Name_dat'])
df_coord = pd.read_csv('data/df_coordinates.csv')

In [58]:
df_index.rename(columns={"Name_big": "Name"}, inplace=True)
df_bigtable_indexed = pd.merge(df_bigtable, df_index, on="Name", how="inner")

df_coord.rename(columns={"name": "Name_dat"}, inplace=True)
df_big_coord = pd.merge(df_bigtable_indexed, df_coord, on="Name_dat", how="inner")

In [59]:
print(df_bigtable.shape)
print(df_bigtable_indexed.shape)
print(df_index.shape)
print(df_coord.shape)

(6324, 5)
(6307, 6)
(6303, 2)
(6303, 103)


In [67]:
df_big_coord

,Name,Family,Thickness(%),x-Location_of_Max_Thk(%),Camber(%),Name_dat,new_x_coord,new_y_coord,0.0,0.02040816326530612,0.04081632653061224,0.061224489795918366,0.08163265306122448,0.1020408163265306,0.12244897959183673,0.14285714285714285,0.16326530612244897,0.18367346938775508,0.2040816326530612,0.22448979591836732,0.24489795918367346,0.26530612244897955,0.2857142857142857,0.3061224489795918,0.32653061224489793,0.3469387755102041,0.36734693877551017,0.3877551020408163,0.4081632653061224,0.42857142857142855,0.44897959183673464,0.4693877551020408,0.4897959183673469,0.5102040816326531,0.5306122448979591,0.5510204081632653,0.5714285714285714,0.5918367346938775,0.6122448979591836,0.6326530612244897,0.6530612244897959,0.673469387755102,0.6938775510204082,0.7142857142857142,0.7346938775510203,0.7551020408163265,0.7755102040816326,0.7959183673469387,0.8163265306122448,0.836734693877551,0.8571428571428571,0.8775510204081632,0.8979591836734693,0.9183673469387754,0.9387755102040816,0.9591836734693877,0.9795918367346939,1.0,0.0.1,0.02040816326530612.1,0.04081632653061224.1,0.061224489795918366.1,0.08163265306122448.1,0.1020408163265306.1,0.12244897959183673.1,0.14285714285714285.1,0.16326530612244897.1,0.18367346938775508.1,0.2040816326530612.1,0.22448979591836732.1,0.24489795918367346.1,0.26530612244897955.1,0.2857142857142857.1,0.3061224489795918.1,0.32653061224489793.1,0.3469387755102041.1,0.36734693877551017.1,0.3877551020408163.1,0.4081632653061224.1,0.42857142857142855.1,0.44897959183673464.1,0.4693877551020408.1,0.4897959183673469.1,0.5102040816326531.1,0.5306122448979591.1,0.5510204081632653.1,0.5714285714285714.1,0.5918367346938775.1,0.6122448979591836.1,0.6326530612244897.1,0.6530612244897959.1,0.673469387755102.1,0.6938775510204082.1,0.7142857142857142.1,0.7346938775510203.1,0.7551020408163265.1,0.7755102040816326.1,0.7959183673469387.1,0.8163265306122448.1,0.836734693877551.1,0.8571428571428571.1,0.8775510204081632.1,0.8979591836734693.1,0.9183673469387754.1,0.9387755102040816.1,0.9591836734693877.1,0.9795918367346939.1,1.0.1
0,63A108 MOD C,NASA,7.7,30.1,0.6,a63a108c.DAT,[0. 0.02040816 0.04081633 0.06122449 0...,[ 0. -0.01572449 -0.01982653 -0.022561...,0.000000,-0.015724,-0.019827,-0.022561,-0.024612,-0.026663,-0.028031,-0.029398,-0.030082,-0.030765,-0.031449,-0.032133,-0.032816,-0.032816,-0.032816,-0.032816,-0.032816,-0.032816,-0.032816,-0.032816,-0.032133,-0.031449,-0.030765,-0.030082,-0.029398,-0.028714,-0.028031,-0.026663,-0.025980,-0.024612,-0.023245,-0.022561,-0.021194,-0.019827,-0.018459,-0.017092,-0.015724,-0.015041,-0.013673,-0.012306,-0.010939,-0.009571,-0.008204,-0.007520,-0.006153,-0.004786,-0.003418,-0.002051,-0.000684,0.00000,0.0,0.021478,0.027742,0.032216,0.034901,0.036691,0.038481,0.039376,0.041165,0.041165,0.042060,0.042955,0.042955,0.042955,0.042955,0.042955,0.042955,0.042955,0.042955,0.042955,0.042060,0.042060,0.041165,0.040270,0.040270,0.039376,0.038481,0.037586,0.036691,0.035796,0.034006,0.033111,0.032216,0.030427,0.028637,0.027742,0.025952,0.024162,0.023267,0.021478,0.019688,0.018793,0.017003,0.015213,0.014318,0.012529,0.010739,0.009844,0.008054,NaN
1,A18,Uncategorized,7.3,30.1,3.2,a18.DAT,[0. 0.02040816 0.04081633 0.06122449 0...,[ nan -0.01334918 -0.01661122 -0.017834...,NaN,-0.013349,-0.016611,-0.017834,-0.018650,-0.018650,-0.018650,-0.018650,-0.018650,-0.018650,-0.018650,-0.018650,-0.018650,-0.018650,-0.018650,-0.018650,-0.018650,-0.018650,-0.018650,-0.018650,-0.018650,-0.018650,-0.018650,-0.018650,-0.018650,-0.018650,-0.018650,-0.018650,-0.018650,-0.018650,-0.018650,-0.018650,-0.018650,-0.018650,-0.018650,-0.018650,-0.018650,-0.018650,-0.018650,-0.018650,-0.018650,-0.018650,-0.018650,-0.018650,-0.018650,-0.018650,-0.018650,-0.018650,-0.018650,-0.01865,NaN,0.017937,0.025949,0.032359,0.038768,0.043576,0.046781,0.049986,0.051588,0.054793,0.056395,0.059600,0.061203,0.062805,0.062805,0.064408,0.064408,0.064408,0.064408,0.064408,0.064408,0.064408,0.062805,0.062805,0.061203,0.059600,0.057998,0.056395,0.054

### 2. Exceptions to treat

In [61]:
df_big_coord.Name.value_counts().sort_values(ascending=False)

Name
FX 66-17AII-182       4
BOEING 737 MIDSPAN    4
MH 102 16.99%         1
A6016-S               1
A6020-S               1
                     ..
ZONE-46               1
YS-915                1
USA-35B               1
USA 98                1
ZONE-52               1
Name: count, Length: 6301, dtype: int64

In [62]:
df_big_coord[df_big_coord['Name'] == 'FX 66-17AII-182']

,Name,Family,Thickness(%),x-Location_of_Max_Thk(%),Camber(%),Name_dat,new_x_coord,new_y_coord,0.0,0.02040816326530612,...,0.8163265306122448.1,0.836734693877551.1,0.8571428571428571.1,0.8775510204081632.1,0.8979591836734693.1,0.9183673469387754.1,0.9387755102040816.1,0.9591836734693877.1,0.9795918367346939.1,1.0.1
592,FX 66-17AII-182,Wortmann,18.8,35.8,3.7,fx6617ai.DAT,[0. 0.02040816 0.04081633 0.06122449 0...,[ 0. -0.01888 -0.02714 -0.03186 ...,0.0,-0.018880,...,0.042664,0.037331,0.031998,0.029332,0.023999,0.018666,0.013333,0.008000,0.005333,NaN
593,FX 66-17AII-182,Wortmann,18.2,35.8,3.8,fx6617ai.DAT,[0. 0.02040816 0.04081633 0.06122449 0...,[ 0. -0.01888 -0.02714 -0.03186 ...,0.0,-0.018880,...,0.042664,0.037331,0.031998,0.029332,0.023999,0.018666,0.013333,0.008000,0.005333,NaN
594,FX 66-17AII-182,Wortmann,18.8,35.8,3.7,fx6617a2.DAT,[0. 0.02040816 0.04081633 0.06122449 0...,[ 0. -0.01628878 -0.02280429 -0.028233...,0.0,-0.016289,...,0.042113,0.039481,0.034217,0.028952,0.023688,0.018424,0.015792,0.010528,0.005264,0.0
595,FX 66-17AII-182,Wortmann,18.2,35.8,3.8,fx6617a2.DAT,[0. 0.02040816 0.04081633 0.06122449 0...,[ 0. -0.01628878 -0.02280429 -0.028233...,0.0,-0.016289,...,0.042113,0.039481,0.034217,0.028952,0.023688,0.018424,0.015792,0.010528,0.005264,0.0


In [63]:
df_big_coord[df_big_coord['Name'] == 'BOEING 737 MIDSPAN']

,Name,Family,Thickness(%),x-Location_of_Max_Thk(%),Camber(%),Name_dat,new_x_coord,new_y_coord,0.0,0.02040816326530612,...,0.8163265306122448.1,0.836734693877551.1,0.8571428571428571.1,0.8775510204081632.1,0.8979591836734693.1,0.9183673469387754.1,0.9387755102040816.1,0.9591836734693877.1,0.9795918367346939.1,1.0.1
154,BOEING 737 MIDSPAN,Boeing,10.0,37.7,1.4,b737c.DAT,[0. 0.02040816 0.04081633 0.06122449 0...,[ 0. -0.00994898 -0.0122449 -0.014540...,0.0,-0.009949,...,0.024390,0.021822,0.019255,0.016688,0.014120,0.011553,0.008986,0.005135,0.002567,NaN
155,BOEING 737 MIDSPAN,Boeing,12.6,28.2,0.7,b737c.DAT,[0. 0.02040816 0.04081633 0.06122449 0...,[ 0. -0.00994898 -0.0122449 -0.014540...,0.0,-0.009949,...,0.024390,0.021822,0.019255,0.016688,0.014120,0.011553,0.008986,0.005135,0.002567,NaN
156,BOEING 737 MIDSPAN,Boeing,10.0,37.7,1.4,b737b.DAT,[0. 0.02040816 0.04081633 0.06122449 0...,[ 0. -0.0216 -0.0288 -0.0348 ...,0.0,-0.021600,...,0.014773,0.013229,0.010139,0.007049,0.005504,0.002414,-0.000676,-0.003765,-0.006855,-0.0084
157,BOEING 737 MIDSPAN,Boeing,12.6,28.2,0.7,b737b.DAT,[0. 0.02040816 0.04081633 0.06122449 0...,[ 0. -0.0216 -0.0288 -0.0348 ...,0.0,-0.021600,...,0.014773,0.013229,0.010139,0.007049,0.005504,0.002414,-0.000676,-0.003765,-0.006855,-0.0084


# Data Pre-processing

# Data Modelling